In [ ]:
# %load ./init.ipy
%reload_ext autoreload
%autoreload 2

# Builtin packages
from importlib import reload
import logging
import os
from pathlib import Path
import sys
import warnings

# standard secondary packages
import astropy as ap
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.stats
import tqdm.notebook as tqdm

# development packages
import kalepy as kale
import kalepy.utils
import kalepy.plot

# --- Holodeck ----
import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR
import holodeck.gravwaves
import holodeck.evolution
import holodeck.population

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})

# Load log and set logging level
log = holo.log
log.setLevel(logging.INFO)

In [ ]:
amp = 10.0
xref = 1.0
g1 = 0.57
g2 = -0.23
x0 = 1e-3

def func(xx):
    lo = (xx < xref)
    hi = ~lo
    yy = np.zeros_like(xx)
    yy[lo] = amp * np.power(xx[lo]/xref, g1)
    yy[hi] = amp * np.power(xx[hi]/xref, g2)
    return yy

def integral(xx):
    lo = (xx < xref)
    hi = ~lo
    
    yy = np.zeros_like(xx)
    yy[lo] = (amp / (g1+1)) * ((xx[lo] * (xx[lo]/xref)**g1) - (x0 * (x0/xref)**g1))    
    
    floor = (amp / (g1+1)) * (xref - x0 * (x0/xref)**g1)
    yy[hi] = floor + (amp / (g2+1)) * ((xx[hi] * (xx[hi]/xref)**g2) - xref)
    return yy    

xx = np.logspace(np.log10(x0), 3, 1000)
yy = func(xx)
zz = integral(xx)
print(f"{zz[-1]:.4e}")

fig, ax = plt.subplots(figsize=[15, 10])
ax.set(xscale='log', yscale='log')
ax.grid(True)
tw = ax.twinx()
tw.set(yscale='log')
cc, = ax.plot(xx, yy)
tw.plot(xx, zz, color=cc.get_color(), ls='--')


NUM = 7
aa = np.logspace(np.log10(x0), 3, NUM)
bb = func(aa)
da = np.diff(aa)
b1l = np.cumsum(bb[:-1]*da)
b1r = np.cumsum(bb[1:]*da)
b1 = 0.5 * (b1l + b1r)
tw.plot([], [])
cc, = tw.plot(aa[1:], b1)

b2 = utils.trapz_loglog(bb, aa)
print(aa.shape, bb.shape, b2.shape)
tw.plot(aa[1:], b2)

plt.show()
    

In [ ]:
ztrue = 10.0 ** np.interp(np.log10(aa[1:]), np.log10(xx), np.log10(zz))
d1 = (b1 - ztrue) / ztrue
d2 = (b2 - ztrue) / ztrue

fig, ax = plt.subplots(figsize=[10, 6])
ax.set(xscale='log', yscale='log')

for dd in [d1, d2]:
    plt.plot(aa[1:], np.fabs(dd))
    
plt.show()

# Mathematical & Numerical Functions

## `any` and `all` - short-circuiting versions of corresponding `numpy` functions.

In [ ]:
import datetime

func_names = ['any', 'all']
func_funcs = [utils.any, utils.all]
check_funcs = [np.any, np.all]
def_value = [False, True]

NUM = 1e4
TRIES = 20

for name, func, check, val in zip(func_names, func_funcs, check_funcs, def_value):
    print(f"{name=}")
    sel = [0, 1, 2]
    for ss in sel:
        test = np.ones(int(NUM), dtype=bool) * val
        test[np.random.choice(test.size, ss, replace=False)] = (not val)
        times_func = np.zeros(TRIES)
        times_check = np.zeros(TRIES)
        for rr in range(TRIES):
            beg = datetime.datetime.now()
            ff = func(test)
            times_func[rr] = (datetime.datetime.now() - beg).total_seconds()
            
            beg = datetime.datetime.now()
            cc = check(test)
            times_check[rr] = (datetime.datetime.now() - beg).total_seconds()

            assert func != check
            assert ff == cc

        print(f"{ss=}")
        print(f"func : {times_func.mean():.4e} ± {times_func.std():.4e}")
        print(f"check: {times_check.mean():.4e} ± {times_check.std():.4e}")
        

# Generate values for testing GW Methods

In [ ]:
def pout(keys, vals):
    for kk, vv in zip(keys, vals):
        try:
            print(kk, "=", "[" + ", ".join([f"{xx:.8e}" for xx in vv]) + "]")
        except:   # nocov
            print(kk, vv)
            raise
        
    return

SIZE = 5
freq = 1.0 / YR
time = GYR

np.random.seed(12543)
m1 = np.random.uniform(6, 9, SIZE)
m2 = np.random.uniform(6, 9, SIZE)
m1, m2 = [(10.0 ** mm) * MSOL for mm in [m1, m2]]
mt = m1 + m2
rs = utils.schwarzschild_radius(mt)
ee = np.random.uniform(0.0, 1.0, SIZE-1)
ee = np.concatenate([ee, [0.999]])
aa = rs * (10.0 ** np.random.uniform(0, 4, SIZE))
dc = (10.0 ** np.random.uniform(1, 9, SIZE)) * PC

keys = ['m1', 'm2', 'aa', 'ee', 'dc']
vals = [m1, m2, aa, ee, dc]
pout(keys, vals)

mc = utils.chirp_mass(m1, m2)
hs = utils.gw_strain_source(mc, dc, freq)
gwlum = utils.gw_lum_circ(mc, freq)
dedt = utils.gw_dedt(m1, m2, aa, ee)
dade = utils.gw_dade(m1, m2, aa, ee)
dadt = utils.gw_hardening_rate_dadt(m1, m2, aa, ee)
dfdt, _ = utils.gw_hardening_rate_dfdt(m1, m2, freq, ee)
tau = utils.gw_hardening_timescale_freq(mc, freq)

print()
keys = ['mc', 'hs', 'gwlum', 'dedt', 'dade', 'dadt', 'dfdt', 'tau']
vals = [mc, hs, gwlum, dedt, dade, dadt, dfdt, tau]
pout(keys, vals)
